В этом ноутбуке я собираю данные из трех датасетов в один

In [1]:
import pandas as pd
from datasets import Dataset

# Concat Data

Так как тексты от отзывов и вопросов, для решаемой задачи, в общем-то имеют одинаковую природу, эти два датасета просто соединяю между собой, добавля метку responder - бинарный признак со значениями 'Клиент' или 'Магазин', указывающий на истончик записи

In [2]:
feedback_df = pd.read_parquet('__input__/clean_feedback.parquet')

feedback_df

,nmId,text,answer
0,0,25.07.21 получил данный товар. Упаковка была н...,
1,0,"4 раза покупала пасту этой фирмы на подарки, и...","Видимо потому, что с ростом заказов в онлайне ..."
2,0,"Банка, как и у многих, пришла разбитая. К сожа...","Алексей, добрый день! Жаль, что баночка была п..."
3,0,"В наш город перестали завозить этот корм, а ко...",
4,0,Вместо льняного семя прислали ореховую пасту и...,"Добрый день. Мы абсолютно с Вами согласны, под..."
...,...,...,...
8270798,9997980,Неплохое платье. На фигуру грушу не очень село...,
8270799,99981867,"Ботинки классные, но пришлось отказаться, так ...",Добрый день! Благодарим вас за заказ и высокую...
8270800,9998234,Беру уже второе такое) очень нравится качество...,Добрый день!\n\nСпасибо Вам за отзыв и обратну...
8270801,9998234,"Покрывало симпатичное, покупаю вашу фирму не в...",Добрый день!\nСпасибо Вам за отзыв и обратную ...


In [3]:
question_df = pd.read_parquet('__input__/clean_questions.parquet')

question_df

,nmId,question,answer
0,1001205,"Здравствуйте! А вес рюкзака подскажите,пожалуй...",Здравствуйте. Вес рюкзака 1080 гр.
1,1001205,Подскажи пожалуйста дно мягкое или твердое?,Здравствуйте!Дно уплотненно..
2,1001213,Здравствуйте! Какое у него дно? Может ли стоя...,Здравствуйте!Дно рюкзака уплотнено.Вес рюкзака...
3,1002463,"Здравствуйте, а перчатки кожа или кож.зам?",Добрый день.синтетическая кожа
4,1002467,Здравствуйте. Хотела бы уточнить когда у вас ...,Здравствуйте. По поступлению данной модели нет...
...,...,...,...
5494101,1178615,"Здравствуйте, а можно ли узнать артикул этого ...","Здравствуйте, данная модель является реквизито..."
5494102,1178625,Здравствуйте! Будет ли поступление этого товар...,"Здравствуйте, к сожалению, поступления данной ..."
5494103,1178626,Здравствуйте! А пуговки не идут в комплекте до...,"Здравствуйте. К сожалению, ответить на Ваш во..."
5494104,1178626,здравствуйте! ожидается ли поступление товара ...,"Здравствуйте, к сожалению, поступлений данной ..."


Из-за того, что у меня в данный момент не ресурсов обработать такой датасет, было принято решение случайным образом выбрать 1 млн записей

In [4]:
feedback_df = feedback_df.sample(500000)
question_df = question_df.sample(500000)

Данные планируется привести сначала к схеме:
* nmId (int64) - Id продукта
* text (string) - Текст вопроса/отзыва клиента или магазина
* type (class: {Вопрос, Отзыв}) - Текст принадлежит датасету вопросов или отзывов
* responder (class: {Клиент, Магазин}) - Оставил ли текст Клиент или это ответ магазина

In [5]:
def split_qa(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename({
        'question': 'text'
    }, axis=1)

    q_df = df[[
        'nmId',
        'text'
    ]].dropna().drop_duplicates()
    a_df = df[[
        'nmId',
        'answer'
    ]].dropna().drop_duplicates()

    a_df = a_df.rename({
        'answer': 'text'
    }, axis=1)

    q_df['responder'] = 'Клиент'
    a_df['responder'] = 'Магазин'

    return pd.concat([
        q_df,
        a_df
    ])

In [6]:
feedback_df = split_qa(feedback_df)

In [7]:
question_df = split_qa(question_df)

In [8]:
feedback_df['type'] = 'Отзыв'
question_df['type'] = 'Вопрос'

In [9]:
all_df = pd.concat([
    feedback_df,
    question_df
])

all_df

,nmId,text,responder,type
7715824,168406783,Заказывала пенку с алое вера для проблемной ко...,Клиент,Отзыв
1516882,3767664,"Стильный пиджачок! Хлопок, тонкий. На бирке н...",Клиент,Отзыв
4732095,42589420,"На свой 56 перезаказала 58 размер, вот он подо...",Клиент,Отзыв
125535,10513374,"Ручка красивая, крупная, смотрится хорошо и за...",Клиент,Отзыв
6041942,100287701,Пришла водолазка с браком. Наверное шили её сл...,Клиент,Отзыв
...,...,...,...,...
1237489,3706159,"Добрый день!Длина стельки 26,5 см.",Магазин,Вопрос
4924538,8132501,Здравствуйте! Поставщик в праве сам регулирова...,Магазин,Вопрос
866208,3073284,Доброе утро! Вероятно Вам пришел брак. Спасибо...,Магазин,Вопрос
1569776,4264147,Добрый день! \n\nАрт. 4264148,Магазин,Вопрос


In [10]:
del feedback_df
del question_df

# Обогащение данных

Теперь мы можем обогатить наши тексты информацией по продукту, к которому они оставлены

In [11]:
products_df = pd.read_parquet(
    './__input__/clean_products.parquet',
    columns=[
        'nm_id',
        'imt_name',
        'subj_name',
        'subj_root_name',
        'nm_colors_names',
        'description',
        'brand_name'
    ]
)

products_df = products_df.rename({
    'nm_id': 'nmId',
    'imt_name': 'product_name',
    'subj_name': 'product_category_2',
    'subj_root_name': 'product_category_1',
    'nm_colors_names': 'product_color',
    'description': 'product_description',
    'brand_name': 'product_brand',
}, axis=1)

products_df.head()

,nmId,product_name,product_category_2,product_category_1,product_color,product_description,product_brand
0,13349977,Очки TRUESPIN Candor Art,Солнцезащитные очки,Аксессуары,белый,,True Spin
1,13350015,Капри,Капри,Одежда,белый,,LAFEI-NIER
2,13350016,Джинсы,Джинсы,Одежда,голубой,,LAFEI-NIER
3,13350019,Топ,Топы,Одежда,бежевый,,LAFEI-NIER
4,13350027,Жакет/приталенный /большие размеры,Жакеты,Одежда,голубой,Классический приталенный пиджак из стрейч ткан...,LAFEI-NIER


In [12]:
all_df = all_df.merge(products_df, on='nmId', how='left')

all_df.head()

,nmId,text,responder,type,product_name,product_category_2,product_category_1,product_color,product_description,product_brand
0,168406783,Заказывала пенку с алое вера для проблемной ко...,Клиент,Отзыв,NaN,NaN,NaN,NaN,NaN,NaN
1,3767664,"Стильный пиджачок! Хлопок, тонкий. На бирке н...",Клиент,Отзыв,Пиджак,Пиджаки,Одежда,синий,,Sisley Young
2,42589420,"На свой 56 перезаказала 58 размер, вот он подо...",Клиент,Отзыв,Платье для дома,Платья,Одежда,"синий, желтый, белый, голубой",Платье повседневное домашнее хлопковое большие...,Милослава
3,10513374,"Ручка красивая, крупная, смотрится хорошо и за...",Клиент,Отзыв,"Ручка керамическая для мебели ""Ливанский узор""...",Ручки мебельные,Мебель,,Самая обычная мебель с непритязательным дизайн...,BlumenHaus
4,100287701,Пришла водолазка с браком. Наверное шили её сл...,Клиент,Отзыв,Базовая водолазка с горлом,Водолазки,Одежда,черный,Водолазка женская – модный и универсальный э...,PUSTOVETOVAAV


Процент ненайденный nmId в продуктах

In [17]:
print(f'{len(all_df[all_df['product_brand'].isna()]) / len(all_df):%}')

2.670211%


In [18]:
all_df = all_df.dropna(subset=['product_brand'])

all_df.to_parquet('./__input__/dataset.parquet')

# Сохраняю итоговый на HF

In [3]:
ds: Dataset = Dataset.from_parquet('./__input__/dataset.parquet') # type: ignore
ds.train_test_split(
    test_size=0.25,
    shuffle=True
)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['nmId', 'text', 'responder', 'type', 'product_name', 'product_category_2', 'product_category_1', 'product_color', 'product_description', 'product_brand', '__index_level_0__'],
        num_rows: 1435746
    })
    test: Dataset({
        features: ['nmId', 'text', 'responder', 'type', 'product_name', 'product_category_2', 'product_category_1', 'product_color', 'product_description', 'product_brand', '__index_level_0__'],
        num_rows: 478583
    })
})

In [5]:
ds.push_to_hub('Roaoch/urfu_ecom_wb')

Uploading the dataset shards:   0%|          | 0/7 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/274 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/274 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/274 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/274 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/274 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/274 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/274 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Roaoch/urfu_ecom_wb/commit/989a1d9b420a9e76b8e5390bd6b9b13980119b40', commit_message='Upload dataset', commit_description='', oid='989a1d9b420a9e76b8e5390bd6b9b13980119b40', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Roaoch/urfu_ecom_wb', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Roaoch/urfu_ecom_wb'), pr_revision=None, pr_num=None)